In [1]:
from matplotlib import animation,rc
rc('animation', html='html5')
%matplotlib inline

In [2]:
def animate_func(X, solutions, const, t_start=0, speed=None, xlim=(0,1), ylim=(-0.2, 1.2)):
    x = X
    y = solutions
    X_const, y_const = const
    fig = plt.figure(figsize=(12, 8), dpi= 80, facecolor='w', edgecolor='k')
    ax = fig.add_subplot(111, xlim=xlim, ylim=ylim)
    line, = plt.plot([], [], '-', lw=2) 
    plt.plot(X_const, y_const, '-', lw=2) 
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
    #line, = plt.plot([], [])
    def animate(i):
        interval = (1./float(len(y)))
        line.set_data(x[i], y[i])
        time_text.set_text('time = %.3f' % (i*len(y)/float(len(y))))
        return line, time_text, 
    if speed is not None:
        anim = animation.FuncAnimation(fig, animate, frames=len(y), interval=speed, blit=True)
    else:
        anim = animation.FuncAnimation(fig, animate, frames=len(y), interval=50, blit=True)
    plt.close(anim._fig)
    return anim
    #a = animation.FuncAnimation(f, animate, frames=y.size, interval=20)
    #plt.show()

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from math import floor
import seaborn

In [4]:
def exact(u0, a, xmin, xmax, t_final):
    x = np.linspace(xmin, xmax, u0.size+1)[:-1]
    distance = a * t_final
    roll = int((distance - floor(distance)) * u0.size)
    #plt.plot(x, np.roll(u0, roll), label="Exact solution")
    return x, np.roll(u0, roll)

In [5]:
def upwind(u0, a, xmin, xmax, t_final, nt):
    """ Solve the advection equation with periodic
    boundary conditions on the interval [xmin, xmax]
    using the upwind finite volume scheme.
    Use u0 as the initial conditions.
    a is the constant from the PDE.
    Use the size of u0 as the number of nodes in
    the spatial dimension.
    Let nt be the number of spaces in the time dimension
    (this is the same as the number of steps if you do
    not include the initial state).
    Plot and show the computed solution along
    with the exact solution. """
    dt = float(t_final) / nt
    # Since we are doing periodic boundary conditions,
    # we need to divide by u0.size instead of (u0.size - 1).
    dx = float(xmax - xmin) / u0.size
    lambda_ = a * dt / dx
    u = u0.copy()
    for j in range(nt):
        # The Upwind method. The np.roll function helps us
        # account for the periodic boundary conditions.
        u -= lambda_ * (u - np.roll(u, 1))
    # Get the x values for the plots.
    x = np.linspace(xmin, xmax, u0.size+1)[:-1]
    # Plot the computed solution.
    #plt.plot(x, u, label="Upwind Method")
    # Find the exact solution and plot it.
    # Show the plot with the legend.
    return x, u

def u(x):
    u = np.exp(-(x - .3)**2 / .005)
    arr = (.6 < x) & (x < .7 )
    u[arr] += 1.
    return u
uPrime = u(np.linspace(0., 1., 10**3)[:-1])

In [6]:
X = []
Y = []
for nx in range(30, 250, 10):
    nt = nx * 3 // 2
    x = np.linspace(0., 1., nx+1)[:-1]
    u0 = u(x)
    # Run the simulation.

    x, sol = upwind(u0, 1.2, 0, 1, 1.2, nt)
    X.append(x)
    Y.append(sol)
    
const = exact(uPrime, 1.2, 0, 1, 1.2)


animate_func(X, Y, const, speed=500)

In [7]:
def lax_wendroff(u0, a, xmin, xmax, t_final, nt):
    dt = float(t_final) / nt
    dx = float(xmax - xmin) / u0.size
    lambda_ = a * dt / dx
    u = u0.copy()
    for j in range(nt):
        mi0 = (np.roll(u,-1)-u)/dx
        mi1 = np.roll(mi0, 1)
        f0 = a*(np.roll(u,1)+(mi1/2)*(dx-a*dt))
        f1 = a*(u+(mi0/2)*(dx-a*dt))
        u -= (dt/dx)*(f1-f0)
    x = np.linspace(xmin, xmax, u0.size+1)[:-1]
    return x, u

In [8]:
X = []
Y = []

for nx in range(30, 250, 10):
    nt = nx * 3 // 2
    x = np.linspace(0., 1., nx+1)[:-1]
    u0 = u(x)
    # Run the simulation.

    x, sol = lax_wendroff(u0, 1.2, 0, 1, 1.2, nt)
    X.append(x)
    Y.append(sol)
    
const = exact(uPrime, 1.2, 0, 1, 1.2)
animate_func(X, Y, const, speed=500)

animate_func(X, Y, const, speed=500)

In [13]:
def minmod(u0, a, xmin, xmax, t_final, nt):
    def _minmod(a,b):
        if a*b <0:
            return 0
        else:
            if abs(a) < abs(b):
                return a
            else:
                return b
    _minmod = np.vectorize(_minmod)

    dt = float(t_final) / nt
    dx = float(xmax - xmin) / u0.size
    lambda_ = a * dt / dx
    u = u0.copy()
    for j in range(nt):
        mi0 = _minmod((np.roll(u,-1)-u)/dx, (u-np.roll(u,1))/dx)
        mi1 = _minmod((u-np.roll(u,1))/dx, (np.roll(u, 1)-np.roll(u, 2))/dx)
        f0 = a*(np.roll(u,1)+(mi1/2)*(dx-a*dt))
        f1 = a*(u+(mi0/2)*(dx-a*dt))
        u -= (dt/dx)*(f1-f0)
    x = np.linspace(xmin, xmax, u0.size+1)[:-1]
    return x, u

In [14]:
X = []
Y = []

for nx in range(30, 250, 10):
    nt = nx * 3 // 2
    x = np.linspace(0., 1., nx+1)[:-1]
    u0 = u(x)
    # Run the simulation.

    x, sol = minmod(u0, 1.2, 0, 1, 1.2, nt)
    X.append(x)
    Y.append(sol)
    
const = exact(uPrime, 1.2, 0, 1, 1.2)
animate_func(X, Y, const, speed=500)

animate_func(X, Y, const, speed=500)